# **New Variable Processing**
`BIOMASS, SFC_CHL, SFC_FED, SFC_IRR`

`BIOMASS` consists of `sfc_ndi`, `sfc_nlg_diatoms`, `sfc_nlg_nondiatoms`, and `sfc_nsm`

In [1]:
import os
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="invalid value encountered in reduce")
warnings.filterwarnings("ignore", message="Degrees of freedom <= 0 for slice.")

import xarray as xr
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

from PIL import Image

%load_ext autoreload
%autoreload 2
import hmei.processing as pcs
import hmei.visualization as vis

In [1]:
d = '/local/projects/so_predict/esm2m_froelicher/CTRL/'
# pcs.dir_inspect(d+'BIOMASS')

# xr.open_dataset(d+'BIOMASS/sfc_ndi_0001.nc')

# Revamped Processing Functions

In [2]:
def comp_gc_metrics(
    ds, variable, resize=False, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/CTRL/'
    subdir = variable.upper()+'/'
    reg = 'global'
    
    da = ds[variable.lower()]
    
    if resize:
        da = pcs.SouthernOcean_resize(da, time=True)
        reg = 'so'
    
    ## global climatology
    clim = da.groupby('time.month').mean(dim='time')
    clim.name = variable.lower()+'_clim'
    clim.attrs = da.attrs
    clim.attrs['description'] = 'monthly climatology'
    
    clim_name = variable.lower()+'_gc_'+reg+'_clim.nc'
    clim = clim.to_dataset()
    clim.attrs['name'] = clim_name
    
    ## global monthly anomaly
    anom = da.groupby('time.month') - clim[variable.lower()+'_clim']
    anom.name = variable.lower()+'_anom'
    anom.attrs = da.attrs
    anom.attrs['description'] = 'monthly anomaly'
    
    anom_name = variable.lower()+'_gc_'+reg+'_monthly_anom.nc'
    anom = anom.to_dataset()
    anom.attrs['name'] = anom_name
    
    ## global monthly variance
    var = anom[variable.lower()+'_anom'].groupby('time.month').var(dim='time')
    var.name = variable.lower()+'_var'
    var.attrs = da.attrs
    var.attrs['units'] = '( '+da.attrs['units']+' )^2'
    var.attrs['description'] = 'variance of monthly anomaly'
    
    var_name = variable.lower()+'_gc_'+reg+'_monthly_var.nc'
    var = var.to_dataset()
    var.attrs['name'] = var_name
    
    if save:
        clim.to_netcdf(writedir+subdir+clim_name)
        anom.to_netcdf(writedir+subdir+anom_name)
        var.to_netcdf(writedir+subdir+var_name)
        
        print(writedir+subdir+clim_name)
        print(writedir+subdir+anom_name)
        print(writedir+subdir+var_name)
        
    if not save:
        print(writedir+subdir+clim_name)
        print(writedir+subdir+anom_name)
        print(writedir+subdir+var_name)
        return clim,anom,var

In [3]:
def comp_annual_mean(
    ds, var, reg, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
    
    ## get ocean model grid information
    ocean_grid = xr.open_dataset('/local/projects/so_predict/esm2m_froelicher/GRID/ocean.static.nc')
    
    ## get grid-cell area for the ocean only
    total_area = xr.where(np.isnan(ds[var][0]), np.nan, ocean_grid['area_t'])
    
    ## global mean
    if reg == 'global':
        area_sum = total_area.sum(dim={'xt_ocean', 'yt_ocean'})
        
        if var == 'SIE_area' or var == 'SIV_area':
            annual_mean = ds[var].groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
        else:
            annual_mean = (ds[var] * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
        
        annual_mean.name = 'Global'
        annual_mean.attrs = ds[var].attrs
        annual_mean.attrs['description'] = 'annual mean'
        annual_mean = annual_mean.to_dataset()
        annual_mean = annual_mean.drop('time')
        
        filename = var.lower()+'_ts_'+reg+'_annual_mean.nc'
        annual_mean.attrs['name'] = filename
        
        if save:
            annual_mean.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
        if not save:
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
            return annual_mean
    
    ## regional mean
    if reg == 'so':
        regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']
        reg_mean = []
        
        for r in regions:
            area = total_area.where(reg_masks[r] == 1)
            area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
            reg_var = ds[var].where(reg_masks[r] == 1)
            
            if var == 'SIE_area' or var == 'SIV_area':
                annual_mean = reg_var.groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
            else:
                annual_mean = (reg_var * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
            
            annual_mean.name = r
            annual_mean.attrs = ds[var].attrs
            annual_mean.attrs['description'] = 'annual mean'
            
            reg_mean.append(annual_mean.copy(deep=True))
            
        annual_mean = xr.merge(reg_mean)
        annual_mean = annual_mean.drop('time')
        
        filename = var.lower()+'_ts_'+reg+'_annual_mean.nc'
        annual_mean.attrs = {}
        annual_mean.attrs['name'] = filename
        
        if save:
            annual_mean.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
        if not save:
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
            return annual_mean

In [4]:
def comp_monthly_mean(
    ds, var, reg, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
    
    ## get ocean model grid information
    ocean_grid = xr.open_dataset('/local/projects/so_predict/esm2m_froelicher/GRID/ocean.static.nc')
    
    ## get grid-cell area for the ocean only
    total_area = xr.where(np.isnan(ds[var][0]), np.nan, ocean_grid['area_t'])
    
    ## global mean
    if reg == 'global':
        area_sum = total_area.sum(dim={'xt_ocean', 'yt_ocean'})
        
        if var == 'SIE_area' or var == 'SIV_area':
            monthly_mean = ds[var].sum(dim={'xt_ocean', 'yt_ocean'})
        else:
            monthly_mean = (ds[var] * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
        
        monthly_mean.name = 'Global'
        monthly_mean.attrs = ds[var].attrs
        monthly_mean.attrs['description'] = 'monthly mean'
        monthly_mean = monthly_mean.to_dataset()
        
        filename = var.lower()+'_ts_'+reg+'_monthly_mean.nc'
        monthly_mean.attrs['name'] = filename
        
        if save:
            monthly_mean.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
        if not save:
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
            return monthly_mean
    
    ## regional mean
    if reg == 'so':
        regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']
        reg_mean = []
        
        for r in regions:
            area = total_area.where(reg_masks[r] == 1)
            area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
            reg_var = ds[var].where(reg_masks[r] == 1)
            
            if var == 'SIE_area' or var == 'SIV_area':
                monthly_mean = reg_var.sum(dim={'xt_ocean', 'yt_ocean'})
            else:
                monthly_mean = (reg_var * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
            
            monthly_mean.name = r
            monthly_mean.attrs = ds[var].attrs
            monthly_mean.attrs['description'] = 'monthly mean'
            
            reg_mean.append(monthly_mean.copy(deep=True))
            
        monthly_mean = xr.merge(reg_mean)
        
        filename = var.lower()+'_ts_'+reg+'_monthly_mean.nc'
        monthly_mean.attrs = {}
        monthly_mean.attrs['name'] = filename
        
        if save:
            monthly_mean.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
        if not save:
            print(writedir+'CTRL/'+var.upper()+'/'+filename)
            return monthly_mean

In [5]:
def comp_monthly_anom(
    var, reg, mean=None, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    if os.path.isfile(writedir+'CTRL/'+var.upper()+'/'+var.lower()+'_ts_'+reg+'_monthly_mean.nc') and mean == None:
        mean = xr.open_dataset(writedir+'CTRL/'+var.upper()+'/'+var.lower()+'_ts_'+reg+'_monthly_mean.nc')
        print('Source: '+var.lower()+'_ts_'+reg+'_monthly_mean.nc')

    ds_anom = mean.copy(deep=True)
    ds_anom['month'] = np.arange(1,3601)
    
    for r in ds_anom.data_vars:
        ds_anom[r] = (('month'), ds_anom[r])
        
    ds_anom = ds_anom.drop('time')
    
    for r in mean.data_vars:
        np_clim = mean[r].groupby('time.month').mean(dim='time').values
        np_mean = mean[r].values

        reg_anom = np.zeros(3600)
        for m in range(0,3600):
            reg_anom[m] = np_mean[m] - np_clim[m%12]

        ds_anom[r] = (('month'), reg_anom)
        ds_anom[r].attrs = mean[r].attrs
        ds_anom[r].attrs['description'] = 'monthly anomaly'
    
    filename = var.lower()+'_ts_'+reg+'_monthly_anom.nc'
    ds_anom.attrs['name'] = filename
    
    if save:
        ds_anom.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
    if not save:
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
        return ds_anom

In [6]:
def comp_monthly_var(
    var, reg, anom=None, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    if os.path.isfile(writedir+'CTRL/'+var.upper()+'/'+var.lower()+'_ts_'+reg+'_monthly_anom.nc') and anom == None:
        anom = xr.open_dataset(writedir+'CTRL/'+var.upper()+'/'+var.lower()+'_ts_'+reg+'_monthly_anom.nc')
        print('Source: '+var.lower()+'_ts_'+reg+'_monthly_anom.nc')
    
    ds_var = anom.copy(deep=True)
    
    if reg == 'global':
        ds_var = ds_var.drop({'month','Global'})
        ds_var['month'] = np.arange(1,13)
        
        np_anom = np.zeros((12,300))
        
        ## group anomaly data by month
        for r in anom.data_vars:
            for m in range(0,3600):           
                np_anom[m%12][int(m/12)] = anom[r].isel(month=m)
        
        ## compute variance
        np_var = np.var(np_anom, axis=1)
        if np.shape(np_var) != (12,):
            raise Exception('variance array is the wrong shape')
            
        ## migrate data from numpy array to dataset
        ds_var['Global'] = (('month'), np_var)
        ds_var['Global'].attrs['name'] = 'variance of monthly anomaly'
    
    if reg == 'so':
        ds_var = ds_var.drop({'month','SouthernOcean','Weddell','Indian','WestPacific','Ross','AmundBell'})
        ds_var['month'] = np.arange(1,13)
        
        np_anom = np.zeros((len(anom.data_vars),12,300))
        
        ## group anomaly data by month
        for (r,i) in zip(anom.data_vars,range(6)):
            for m in range(0,3600):           
                np_anom[i][m%12][int(m/12)] = anom[r].isel(month=m)
        
        ## compute variance
        np_var = np.var(np_anom, axis=2)
        if np.shape(np_var) != (6,12):
            raise Exception('variance array is the wrong shape')
        
        ## migrate data from numpy array to dataset
        for (r,i) in zip(anom.data_vars, range(6)):
            ds_var[r] = (('month'), np_var[i])
            ds_var[r].attrs['name'] = 'variance of monthly anomaly'
    
    filename = var.lower()+'_ts_'+reg+'_monthly_var.nc'
    ds_var.attrs['name'] = filename
    
    if save:
        ds_var.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
    if not save:
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
        return ds_var

In [7]:
def comp_ens_global_mean(
    variables, timescale, save=False, SIE=False, SIV=False):
    
    rootdir = '/local/projects/so_predict/esm2m_froelicher/'
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
    regions = list(reg_masks.data_vars)

    ocean_grid = xr.open_dataset(rootdir+'GRID/ocean.static.nc')
    geolat_t = ocean_grid.geolat_t
    geolon_t = ocean_grid.geolon_t
    
    months = 120

    nT_months = np.arange(1,months+1)
    nT_years = np.arange(1,(months/12)+1)
    nStart = np.arange(1,7)
    nEns = np.empty(0, dtype=str)

    start_yrs = ['0170_0179', '0022_0031', '0064_0073', '0106_0115', '0232_0241', '0295_0304']
    
    biomass_vars = ['sfc_ndi', 'sfc_nlg_diatoms', 'sfc_nlg_nondiatoms', 'sfc_nsm']
    
    ## iterate through all variables
    for var in variables:
        ## update status
        print('\n######## '+var[0]+' ########\n')

        subdir = var[0]+'_ENSEMBLE/'

        ## list with datasets for each start year (includes 40 ensembles)
        yrs_list = []

        ## iterate through the start years
        for (i,yrs) in zip(range(1,7), start_yrs):

            ## list with datasets for each ensemble run (pos/neg, 1-20)
            ens_list = []

            ## reset 'nEns' coord list
            nEns = np.empty(0, dtype=str)

            ## iterate through ensembles
            for sign in ['neg', 'pos']:
                for pert in range(1,21):
                    
                    nEns = np.append(nEns, sign+f'{pert:02}')
                    
                    if var[0] != 'BIOMASS':
                        filename = var[1]+'_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'
                        ds = xr.open_dataset(rootdir+subdir+filename)
                    
                    if var[0] == 'BIOMASS':
                        filename = '_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'
                        ndi = xr.open_dataset(rootdir+subdir+biomass_vars[0]+filename)
                        diatoms = xr.open_dataset(rootdir+subdir+biomass_vars[1]+filename)
                        nondiatoms = xr.open_dataset(rootdir+subdir+biomass_vars[2]+filename)
                        nsm = xr.open_dataset(rootdir+subdir+biomass_vars[3]+filename)

                        ds = ndi.copy(deep=True)
                        ds = ds.rename({'sfc_ndi':'sfc_biomass'})

                        ds['sfc_biomass'] = ndi['sfc_ndi'] + diatoms['sfc_nlg_diatoms'] + nondiatoms['sfc_nlg_nondiatoms'] + nsm['sfc_nsm']
                    
                    ## reassign coord names to ensure continuity between datasets
                    coords = list(ds.coords)
                    if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
                        ds = ds.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})

                    if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
                        ds = ds.rename({'XT_OCEAN':'xt_ocean', 'YT_OCEAN':'yt_ocean', 'TIME':'time'})
                        
                    if (coords.count('xt') == 1) and (coords.count('yt') == 1):
                        ds = ds.rename({'xt':'xt_ocean', 'yt':'yt_ocean'})
                        
                    ## truncate at 120 months for CN_INV
                    if ds['time'].size > months:
                        time = xr.cftime_range(start='0000', periods=months, freq='MS', calendar='noleap')
                        temp = ds.copy(deep=True)
                        ds = ds.drop('time')
                        ds = ds.drop_vars(var[2])
                        ds = ds.assign_coords({'time':time})
                        ds[var[2]] = (('time','yt_ocean','xt_ocean'), temp[var[2]][:months])
                                            
                    ## assign ocean_grid coords for continuity
                    ds = ds.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean})
                    
                    ## remove 'area_t' gridcells on land, keep only the ocean
                    area = xr.where(np.isnan(ds[var[2]][0]), np.nan, ocean_grid['area_t'])

                    ## total ocean area in the region
                    area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
                    
                    ## load and format SIC data for SIV computation
                    if SIV:
                        sie_area = xr.open_dataset(rootdir+'CN_ENSEMBLE/CN_inv_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc')
                        sie_area = sie_area.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})
                        sie_area = sie_area.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean})
        
                        if sie_area['CN_INV'].size > months:
                            temp = sie_area.copy(deep=True)
                            sie_area = sie_area.drop('time')
                            sie_area = sie_area.drop_vars('CN_INV')
                            sie_area['CN_INV'] = (('time','yt_ocean','xt_ocean'), temp['CN_INV'][:months])
                        
                        sie_area = sie_area.assign_coords({'time':ds['time']})
                        sie_area.transpose()
                        sie_area = sie_area['CN_INV'] * area
                        
                    ## threshold SIE at >15% SIC
                    if SIE:
                        gridcell = xr.where(ds[var[2]] > 0.15, 1, 0)

                    if timescale == 'annual':                      
                        if SIE:
                            mean = (gridcell * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
                            
                        elif SIV:
                            mean = (ds[var[2]] * sie_area).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
                        
                        else:                          
                            ## annual mean - compute areal integral for each year
                            ## (1) find annual mean value for each gridcell
                            ## (2) compute areal integral for the annual mean gridcells
                            mean = (ds[var[2]] * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum

                        ## reassign 'time' cftime coord to 'nT' integer coord from 1-10
                        if list(mean.coords).count('time') > 0:
                            mean = mean.drop('time')
                        mean = mean.rename({'year':'nT'})
                        mean = mean.assign_coords({'nT':nT_years})

                    if timescale == 'monthly':
                        if SIE:
                            mean = (gridcell * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'})
                            
                        elif SIV:
                            mean = (ds[var[2]] * sie_area).sum(dim={'xt_ocean', 'yt_ocean'})
                        
                        else:
                            ## monthly mean - compute areal integral for each month
                            mean = (ds[var[2]] * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum

                        ## reassign 'time' cftime coord to 'nT' integer coord from 1-120
                        mean = mean.rename({'time':'nT'})
                        mean = mean.assign_coords({'nT':nT_months})

                    ## rename the array to the region's name
                    mean.name = 'Global'

                    ## convert the array to a dataset
                    this_ens = mean.to_dataset()

                    ## append a copy of the dataset to the ensembles list
                    ens_list.append(this_ens.copy(deep=True))

                    ## update status
                    if (i == 1) and (sign == 'neg') and (pert == 1):
                        print('this_ens.coords = '+str(list(this_ens.coords)))
                        print('this_ens.data_vars = '+str(list(this_ens.data_vars))+'\n')

                    print('o',end='')
                    if pert == 20:
                        print('')

                ## update status
                if sign == 'neg':
                    print('var: '+var[2]+' | nStart: '+str(i)+' | nEns: 20/40')
                if sign == 'pos':
                    print('var: '+var[2]+' | nStart: '+str(i)+' | nEns: 40/40')

            ## concatenate ensemble arrays along new 'nEns' dimension
            these_ens = xr.concat(ens_list, 'nEns')
            these_ens = these_ens.assign_coords({'nEns':nEns}) 

            ## append a copy of the dataset to the start years list
            yrs_list.append(these_ens.copy(deep=True))

            ## update status
            if i == 1:
                print('\nthese_ens.coords = '+str(list(these_ens.coords)))
                print('these_ens.data_vars = '+str(list(these_ens.data_vars))+'\n')

        ## concatenate start years arrays along new 'nStart' dimension
        all_ens = xr.concat(yrs_list, 'nStart')
        all_ens = all_ens.assign_coords({'nStart':start_yrs})

        ## copy attributes to each region data variable
        all_ens['Global'].attrs = mean.attrs

        ## add description
        if SIE:
            filename = 'sie_ens_global_'+timescale+'_mean.nc'
        if SIV:
            filename = 'siv_ens_global_'+timescale+'_mean.nc'
        if not SIE and not SIV:
            filename = var[1].lower()+'_ens_global_'+timescale+'_mean.nc'
        all_ens.attrs['name'] = filename

        print('\nall_ens.coords = '+str(list(all_ens.coords)))
        print('all_ens.data_vars = '+str(list(all_ens.data_vars))+'\n')

        if save:
            if var[0] == 'CN':
                subdir = var[2]+'_ENSEMBLE/'
            if var[0] == 'BIOMASS':
                subdir = var[2].upper()+'_ENSEMBLE/'
            if SIE:
                subdir = 'SIE_ENSEMBLE/'
            if SIV:
                subdir = 'SIV_ENSEMBLE/'
            all_ens.to_netcdf(writedir+subdir+filename)
            print(writedir+subdir+filename)

        elif not save:
            return all_ens

In [8]:
def comp_ens_so_mean(
    variables, timescale, save=False, SIE=False, SIV=False):
    
    rootdir = '/local/projects/so_predict/esm2m_froelicher/'
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
    regions = list(reg_masks.data_vars)

    ocean_grid = xr.open_dataset(rootdir+'GRID/ocean.static.nc')
    geolat_t = ocean_grid.geolat_t
    geolon_t = ocean_grid.geolon_t
    
    months = 120

    nT_months = np.arange(1,months+1)
    nT_years = np.arange(1,(months/12)+1)
    nStart = np.arange(1,7)
    nEns = np.empty(0, dtype=str)

    start_yrs = ['0170_0179', '0022_0031', '0064_0073', '0106_0115', '0232_0241', '0295_0304']
    
    ## iterate through all variables
    for var in variables:
        ## update status
        print('\n######## '+var[0]+' ########\n')

        subdir = var[0]+'_ENSEMBLE/'

        ## list with datasets for each start year (includes 40 ensembles)
        yrs_list = []
        
        biomass_vars = ['sfc_ndi', 'sfc_nlg_diatoms', 'sfc_nlg_nondiatoms', 'sfc_nsm']
        
        ## iterate through the start years
        for (i,yrs) in zip(range(1,7), start_yrs):

            ## list with datasets for each ensemble run (pos/neg, 1-20)
            ens_list = []

            ## reset 'nEns' coord list
            nEns = np.empty(0, dtype=str)

            ## iterate through ensembles
            for sign in ['neg', 'pos']:
                for pert in range(1,21):
                    
                    nEns = np.append(nEns, sign+f'{pert:02}')
                    
                    if var[0] != 'BIOMASS':
                        filename = var[1]+'_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'
                        ds = xr.open_dataset(rootdir+subdir+filename)

                    if var[0] == 'BIOMASS':
                        filename = '_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'
                        ndi = xr.open_dataset(rootdir+subdir+biomass_vars[0]+filename)
                        diatoms = xr.open_dataset(rootdir+subdir+biomass_vars[1]+filename)
                        nondiatoms = xr.open_dataset(rootdir+subdir+biomass_vars[2]+filename)
                        nsm = xr.open_dataset(rootdir+subdir+biomass_vars[3]+filename)

                        ds = ndi.copy(deep=True)
                        ds = ds.rename({'sfc_ndi':'sfc_biomass'})

                        ds['sfc_biomass'] = ndi['sfc_ndi'] + diatoms['sfc_nlg_diatoms'] + nondiatoms['sfc_nlg_nondiatoms'] + nsm['sfc_nsm']
                    
                    ## reassign coord names to ensure continuity between datasets
                    coords = list(ds.coords)
                    if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
                        ds = ds.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})

                    if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
                        ds = ds.rename({'XT_OCEAN':'xt_ocean', 'YT_OCEAN':'yt_ocean', 'TIME':'time'})
                        
                    if (coords.count('xt') == 1) and (coords.count('yt') == 1):
                        ds = ds.rename({'xt':'xt_ocean', 'yt':'yt_ocean'})
                    
                    ## truncate at 120 months for CN_INV
                    if ds['time'].size > months:
                        time = xr.cftime_range(start='0000', periods=months, freq='MS', calendar='noleap')
                        temp = ds.copy(deep=True)
                        ds = ds.drop('time')
                        ds = ds.drop_vars(var[2])
                        ds = ds.assign_coords({'time':time})
                        ds[var[2]] = (('time','yt_ocean','xt_ocean'), temp[var[2]][:months])
                    
                    ## assign ocean_grid coords for continuity
                    ds = ds.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean})
                    
                    ## remove 'area_t' gridcells on land, keep only the ocean
                    glob_area = xr.where(np.isnan(ds[var[2]][0]), np.nan, ocean_grid['area_t'])
                    
                    ## load and format SIC data for SIV computation
                    if SIV:
                        sie_area = xr.open_dataset(rootdir+'CN_ENSEMBLE/CN_inv_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc')
                        sie_area = sie_area.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})
                        sie_area = sie_area.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean})
        
                        if sie_area['CN_INV'].size > months:
                            temp = sie_area.copy(deep=True)
                            sie_area = sie_area.drop('time')
                            sie_area = sie_area.drop_vars('CN_INV')
                            sie_area['CN_INV'] = (('time','yt_ocean','xt_ocean'), temp['CN_INV'][:months])
                        
                        sie_area = sie_area.assign_coords({'time':ds['time']})
                        sie_area.transpose()
                        sie_area = sie_area['CN_INV'] * glob_area
                                    
                    ## compute timeseries for each region
                    reg_list = []
                    for reg in regions:
                        ## regional mask of 'area_t' 
                        area = glob_area.where(reg_masks[reg] == 1)
                        
                        ## total ocean area in the region
                        area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
                        
                        ## regional mask of ensemble run data
                        reg_gridcell = ds[var[2]].where(reg_masks[reg] == 1)
                        
                        ## threshold SIE at >15% SIC
                        if SIE:
                            reg_gridcell = xr.where(reg_gridcell > 0.15, 1, 0)
                        
                        if timescale == 'annual':
                            if SIE:
                                reg_mean = (reg_gridcell * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
                            
                            elif SIV:
                                reg_mean = (sie_area * reg_gridcell).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'})
                            
                            else:
                                ## annual mean - compute area integral for each year
                                ## (1) find annual mean value for each gridcell
                                ## (2) compute areal integral for the annual mean gridcells
                                reg_mean = (reg_gridcell * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
                            
                            ## reassign 'time' cftime coord to 'nT' integer coord from 1-10
                            if list(reg_mean.coords).count('time') > 0:
                                reg_mean = reg_mean.drop('time')
                            reg_mean = reg_mean.rename({'year':'nT'})
                            reg_mean = reg_mean.assign_coords({'nT':nT_years})

                        if timescale == 'monthly':
                            if SIE:
                                reg_mean = (reg_gridcell * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'})
                            
                            elif SIV:
                                reg_mean = (sie_area * reg_gridcell).sum(dim={'xt_ocean', 'yt_ocean'})
                               
                            else:
                                ## monthly mean - compute area integral for each month
                                reg_mean = (reg_gridcell * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
                            
                            ## reassign 'time' cftime coord to 'nT' integer coord from 1-120
                            reg_mean = reg_mean.rename({'time':'nT'})
                            reg_mean = reg_mean.assign_coords({'nT':nT_months})

                        ## rename the array to the region's name
                        reg_mean.name = reg

                        ## append a copy of the array to the regions list
                        reg_list.append(reg_mean.copy(deep=True))

                    ## merge the regions list into one dataset, separate variables for each region
                    this_ens = xr.merge(reg_list)

                    ## append a copy of the dataset to the ensembles list
                    ens_list.append(this_ens.copy(deep=True))

                    ## update status
                    if (i == 1) and (sign == 'neg') and (pert == 1):
                        print('this_ens.coords = '+str(list(this_ens.coords)))
                        print('this_ens.data_vars = '+str(list(this_ens.data_vars))+'\n')

                    print('o',end='')
                    if pert == 20:
                        print('')

                ## update status
                if sign == 'neg':
                    print('var: '+var[2]+' | nStart: '+str(i)+' | nEns: 20/40')
                if sign == 'pos':
                    print('var: '+var[2]+' | nStart: '+str(i)+' | nEns: 40/40')

            ## concatenate ensemble arrays along new 'nEns' dimension
            these_ens = xr.concat(ens_list, 'nEns')
            these_ens = these_ens.assign_coords({'nEns':nEns}) 

            ## append a copy of the dataset to the start years list
            yrs_list.append(these_ens.copy(deep=True))

            ## update status
            if i == 1:
                print('\nthese_ens.coords = '+str(list(these_ens.coords)))
                print('these_ens.data_vars = '+str(list(these_ens.data_vars))+'\n')

        ## concatenate start years arrays along new 'nStart' dimension
        all_ens = xr.concat(yrs_list, 'nStart')
        all_ens = all_ens.assign_coords({'nStart':start_yrs})

        ## copy attributes to each region data variable
        for reg in regions:
            all_ens[reg].attrs = reg_gridcell.attrs

        ## add description
        if SIE:
            filename = 'sie_ens_so_'+timescale+'_mean.nc'
        if SIV:
            filename = 'siv_ens_so_'+timescale+'_mean.nc'
        if not SIE and not SIV:
            filename = var[1].lower()+'_ens_so_'+timescale+'_mean.nc'
        all_ens.attrs['name'] = filename

        print('\nall_ens.coords = '+str(list(all_ens.coords)))
        print('all_ens.data_vars = '+str(list(all_ens.data_vars))+'\n')

        if save:
            if var[0] == 'CN':
                subdir = var[2]+'_ENSEMBLE/'
            if var[0] == 'BIOMASS':
                subdir = var[2].upper()+'_ENSEMBLE/'
            if SIE:
                subdir = 'SIE_ENSEMBLE/'
            if SIV:
                subdir = 'SIV_ENSEMBLE/'
            all_ens.to_netcdf(writedir+subdir+filename)
            print(writedir+subdir+filename)
        elif not save:
            return all_ens

In [9]:
def comp_ens_anom(
    var, timescale, reg, clim=None, save=False):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    subdir = var.upper()+'_ENSEMBLE/'
    
    if clim == None and timescale == 'monthly':
        clim = xr.open_dataset(writedir+'CTRL/'+var.upper()+'/'+var.lower()+'_ts_'+reg+'_monthly_mean.nc')
        clim = clim.groupby('time.month').mean(dim='time')
    
    ## open the ensemble timeseries
    ens_mean = xr.open_dataset(writedir+subdir+var.lower()+'_ens_'+reg+'_'+timescale+'_mean.nc')
    
    ## create list of regions (either SO regions or Global)
    regions = list(ens_mean.data_vars)
        
    ## create a copy of the ensemble timeseries to overwrite with anomaly data
    ens_anom = ens_mean.copy(deep=True)
        
    ## iterate through all start years
    for s in range(ens_anom['nStart'].size):
        ## iterate through each perturbation
        for e in range(ens_anom['nEns'].size):
            ## iterate through all regions
            for r in regions:
                ## create a NumPy array of the ensemble data
                np_mean = ens_mean[r][s,e].values

                ## if SST, adjust units of climatology back to Kelvin
                if var.lower() == 'sst':
                    np_clim = clim[r].values + 273.15
                else:
                    np_clim = clim[r].values 

                ## calculate the anomaly between the ensemble data and the
                ## control run climatology 
                ## (Jan_ens - Jan_ctrl, Feb_ens - Feb_ctrl, etc.)                    
                reg_anom = np.zeros(120)
                for m in range(120):
                    reg_anom[m] = np_mean[m] - np_clim[m%12]

                ens_anom[r][s,e] = reg_anom
                
    ## add description
    if len(regions) > 1:
        filename = var.lower()+'_ens_so_'+timescale+'_anom.nc'
    elif len(regions) == 1:
        filename = var.lower()+'_ens_global_'+timescale+'_anom.nc'
    ens_anom.attrs['name'] = filename
    
    if save:
        ens_anom.to_netcdf(writedir+subdir+filename)
        print(writedir+subdir+filename)
    elif not save:
        return ens_anom

In [15]:
def comp_ppp(
    var, reg, save=False, timescale='monthly'):

    ## import necessary data
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    ## variance of the control simulation for each month
    ## dimensions (month: 12)
    subdir = 'CTRL/'+var.upper()+'/'
    filename = var.lower()+'_ts_'+reg+'_'+timescale+'_var.nc'
    ctrl_var = xr.open_dataset(writedir+subdir+filename)
    
    ## ensemble anomalies
    ## dimensions (nStart: 6, nEns: 40, nT: 120)
    subdir = var.upper()+'_ENSEMBLE/'
    filename = var.lower()+'_ens_'+reg+'_'+timescale+'_anom.nc'
    ens_anom = xr.open_dataset(writedir+subdir+filename)
    
    regions = ctrl_var.data_vars

    ## constants
    N = 6
    M = 40

    ## numerator coefficient
    num_coeff = 1 / (N * (M - 1))

    ## numerator outer sum - across all ensembles
    num_outer_sum = np.zeros((6,120))

    PPP = np.zeros( (len(regions), 120) )

    for (iReg,r) in zip(range(len(regions)), regions):

        for j in range(N):
            ## numerator inner sum - across all ensemble members
            num_inner_sum = np.zeros((40,120))

            ## ensemble mean
            ens_mean = ens_anom[r][j].mean(dim='nEns')

            for i in range(M):
                num_inner_sum[i] = np.square(ens_anom[r][j,i] - ens_mean)

            num_outer_sum[j] = np.sum(num_inner_sum, axis=0)

        ## numerator total sum
        num_sum = np.sum(num_outer_sum, axis=0)

        ## compute PPP
        for m in range(120):
            PPP[iReg][m] = 1 - ( (num_coeff * num_sum[m]) / ctrl_var[r][m%12] )
            
    ## organize PPP data in a Dataset
    PPP_da = []
    for (iReg,r) in zip(range(len(regions)), regions):
        
        PPP_reg = xr.DataArray(
                    data=PPP[iReg],
                    dims=['nT'],
                    coords=dict(
                        nT=np.arange(1,121)
                    ),
                    name=r
                )
        
        PPP_da.append(PPP_reg.copy(deep=True))
    
    PPP_ds = xr.merge(PPP_da)
    PPP_ds.attrs['name'] = var.lower()+'_ts_'+reg+'_'+timescale+'_ppp.nc'
    
    subdir = 'PPP/'+var.upper()+'/'
    filename = var.lower()+'_ts_'+reg+'_'+timescale+'_ppp.nc'
        
    if save:
        PPP_ds.to_netcdf(writedir+subdir+filename)
        print(writedir+subdir+filename)
        
    elif not save:
        print(writedir+subdir+filename)
        return PPP_ds

# Total Surface Biomass

## Control

### Import Raw Data

In [35]:
rootdir = '/local/projects/so_predict/esm2m_froelicher/CTRL/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/CTRL/'
# pcs.dir_inspect(rootdir)

In [3]:
## import raw gridcell data
ndi = xr.open_dataset(rootdir+'BIOMASS/sfc_ndi_0001_0300.nc')
diatoms = xr.open_dataset(rootdir+'BIOMASS/sfc_nlg_diatoms_0001_0300.nc')
nondiatoms = xr.open_dataset(rootdir+'BIOMASS/sfc_nlg_nondiatoms_0001_0300.nc')
nsm = xr.open_dataset(rootdir+'BIOMASS/sfc_nsm_0001_0300.nc')

### Get Total Biomass

In [4]:
## sum all biomass contributors into sfc_biomass variable

sfc_biomass = ndi.copy(deep=True)
sfc_biomass = sfc_biomass.rename({'sfc_ndi':'sfc_biomass'})

sfc_biomass['sfc_biomass'] = ndi['sfc_ndi'] + diatoms['sfc_nlg_diatoms'] + nondiatoms['sfc_nlg_nondiatoms'] + nsm['sfc_nsm']

sfc_biomass['sfc_biomass'].attrs = ndi['sfc_ndi'].attrs
sfc_biomass['sfc_biomass'].attrs['long_name'] = 'Total Surface N'
sfc_biomass['sfc_biomass'].attrs['description'] = '('+ndi['sfc_ndi'].attrs['long_name']+') + ('+diatoms['sfc_nlg_diatoms'].attrs['long_name']+') + ('+nondiatoms['sfc_nlg_nondiatoms'].attrs['long_name']+') + ('+nsm['sfc_nsm'].attrs['long_name']+')'

## save dataset
filename = 'sfc_biomass_gc_global_monthly_mean.nc'
sfc_biomass.to_netcdf(writedir+'SFC_BIOMASS/'+filename)
print(writedir+'SFC_BIOMASS/'+filename)

### Gridcell Metrics

In [32]:
comp_gc_metrics(sfc_biomass, 'sfc_biomass', resize=False, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_global_monthly_var.nc


In [33]:
comp_gc_metrics(sfc_biomass, 'sfc_biomass', resize=True, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_so_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_gc_so_monthly_var.nc


### Timeseries Metrics

In [140]:
comp_annual_mean(sfc_biomass, 'sfc_biomass', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_global_annual_mean.nc


In [141]:
comp_annual_mean(sfc_biomass, 'sfc_biomass', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_so_annual_mean.nc


In [174]:
comp_monthly_mean(sfc_biomass, 'sfc_biomass', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_global_monthly_mean.nc


In [175]:
comp_monthly_mean(sfc_biomass, 'sfc_biomass', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_so_monthly_mean.nc


In [224]:
comp_monthly_anom('sfc_biomass', 'global', save=True)

Source: sfc_biomass_ts_global_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_global_monthly_anom.nc


In [225]:
comp_monthly_anom('sfc_biomass', 'so', save=True)

Source: sfc_biomass_ts_so_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_so_monthly_anom.nc


In [263]:
comp_monthly_var('sfc_biomass', 'global', save=True)

Source: sfc_biomass_ts_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_global_monthly_var.nc


In [264]:
comp_monthly_var('sfc_biomass', 'so', save=True)

Source: sfc_biomass_ts_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_so_monthly_var.nc


In [279]:
pcs.comp_clim('sfc_biomass', 'global', save=True)
pcs.comp_clim('sfc_biomass', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_BIOMASS/sfc_biomass_ts_so_clim.nc


## Ensembles

In [369]:
comp_ens_global_mean([['BIOMASS','sfc_biomass','sfc_biomass']], 'monthly', save=True)


######## BIOMASS ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/h

In [370]:
comp_ens_so_mean([['BIOMASS','sfc_biomass','sfc_biomass']], 'monthly', save=True)


######## BIOMASS ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 20/40
ooooooooooooo

In [371]:
comp_ens_global_mean([['BIOMASS','sfc_biomass','sfc_biomass']], 'annual', save=True)


######## BIOMASS ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/h

In [372]:
comp_ens_so_mean([['BIOMASS','sfc_biomass','sfc_biomass']], 'annual', save=True)


######## BIOMASS ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_biomass | nStart: 6 | nEns: 20/40
ooooooooooooo

In [374]:
comp_ens_anom('sfc_biomass', 'monthly', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_BIOMASS_ENSEMBLE/sfc_biomass_ens_global_monthly_anom.nc


In [375]:
comp_ens_anom('sfc_biomass', 'monthly', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_BIOMASS_ENSEMBLE/sfc_biomass_ens_so_monthly_anom.nc


## PPP

In [23]:
comp_ppp('sfc_biomass', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_BIOMASS/sfc_biomass_ts_global_monthly_ppp.nc


In [24]:
comp_ppp('sfc_biomass', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_BIOMASS/sfc_biomass_ts_so_monthly_ppp.nc


# SFC_CHL

In [34]:
# pcs.dir_inspect(rootdir+'SFC_CHL')

In [304]:
sfc_chl = xr.open_dataset(rootdir+'SFC_CHL/sfc_chl_0001_0300.nc')

In [30]:
comp_gc_metrics(sfc_chl, 'sfc_chl', resize=False, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_global_monthly_var.nc


In [31]:
comp_gc_metrics(sfc_chl, 'sfc_chl', resize=True, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_so_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_gc_so_monthly_var.nc


In [142]:
comp_annual_mean(sfc_chl, 'sfc_chl', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_global_annual_mean.nc


In [143]:
comp_annual_mean(sfc_chl, 'sfc_chl', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_so_annual_mean.nc


In [176]:
comp_monthly_mean(sfc_chl, 'sfc_chl', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_global_monthly_mean.nc


In [177]:
comp_monthly_mean(sfc_chl, 'sfc_chl', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_so_monthly_mean.nc


In [226]:
comp_monthly_anom('sfc_chl', 'global', save=True)

Source: sfc_chl_ts_global_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_global_monthly_anom.nc


In [227]:
comp_monthly_anom('sfc_chl', 'so', save=True)

Source: sfc_chl_ts_so_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_so_monthly_anom.nc


In [265]:
comp_monthly_var('sfc_chl', 'global', save=True)

Source: sfc_chl_ts_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_global_monthly_var.nc


In [266]:
comp_monthly_var('sfc_chl', 'so', save=True)

Source: sfc_chl_ts_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_so_monthly_var.nc


In [280]:
pcs.comp_clim('sfc_chl', 'global', save=True)
pcs.comp_clim('sfc_chl', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_CHL/sfc_chl_ts_so_clim.nc


In [313]:
comp_ens_global_mean([['SFC_CHL','sfc_chl','sfc_chl']], 'monthly', save=True)


######## SFC_CHL ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_C

In [314]:
comp_ens_so_mean([['SFC_CHL','sfc_chl','sfc_chl']], 'monthly', save=True)


######## SFC_CHL ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 40/

In [315]:
comp_ens_global_mean([['SFC_CHL','sfc_chl','sfc_chl']], 'annual', save=True)


######## SFC_CHL ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_C

In [316]:
comp_ens_so_mean([['SFC_CHL','sfc_chl','sfc_chl']], 'annual', save=True)


######## SFC_CHL ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_chl | nStart: 6 | nEns: 40/

In [341]:
comp_ens_anom('sfc_chl', 'monthly', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_CHL_ENSEMBLE/sfc_chl_ens_global_monthly_anom.nc


In [342]:
comp_ens_anom('sfc_chl', 'monthly', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_CHL_ENSEMBLE/sfc_chl_ens_so_monthly_anom.nc


In [25]:
comp_ppp('sfc_chl', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_CHL/sfc_chl_ts_global_monthly_ppp.nc


In [26]:
comp_ppp('sfc_chl', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_CHL/sfc_chl_ts_so_monthly_ppp.nc


# SFC_FED

In [50]:
# pcs.dir_inspect(rootdir+'SFC_FED')

In [39]:
sfc_fed = xr.open_dataset(rootdir+'SFC_FED/sfc_fed_0001_0300.nc')

In [42]:
comp_gc_metrics(sfc_fed, 'sfc_fed', resize=False, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_global_monthly_var.nc


In [43]:
comp_gc_metrics(sfc_fed, 'sfc_fed', resize=True, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_so_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_gc_so_monthly_var.nc


In [144]:
comp_annual_mean(sfc_fed, 'sfc_fed', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_global_annual_mean.nc


In [145]:
comp_annual_mean(sfc_fed, 'sfc_fed', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_so_annual_mean.nc


In [178]:
comp_monthly_mean(sfc_fed, 'sfc_fed', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_global_monthly_mean.nc


In [179]:
comp_monthly_mean(sfc_fed, 'sfc_fed', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_so_monthly_mean.nc


In [228]:
comp_monthly_anom('sfc_fed', 'global', save=True)

Source: sfc_fed_ts_global_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_global_monthly_anom.nc


In [229]:
comp_monthly_anom('sfc_fed', 'so', save=True)

Source: sfc_fed_ts_so_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_so_monthly_anom.nc


In [267]:
comp_monthly_var('sfc_fed', 'global', save=True)

Source: sfc_fed_ts_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_global_monthly_var.nc


In [268]:
comp_monthly_var('sfc_fed', 'so', save=True)

Source: sfc_fed_ts_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_so_monthly_var.nc


In [281]:
pcs.comp_clim('sfc_fed', 'global', save=True)
pcs.comp_clim('sfc_fed', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_FED/sfc_fed_ts_so_clim.nc


In [317]:
comp_ens_global_mean([['SFC_FED','sfc_fed','sfc_fed']], 'monthly', save=True)


######## SFC_FED ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_F

In [318]:
comp_ens_so_mean([['SFC_FED','sfc_fed','sfc_fed']], 'monthly', save=True)


######## SFC_FED ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 40/

In [319]:
comp_ens_global_mean([['SFC_FED','sfc_fed','sfc_fed']], 'annual', save=True)


######## SFC_FED ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_F

In [320]:
comp_ens_so_mean([['SFC_FED','sfc_fed','sfc_fed']], 'annual', save=True)


######## SFC_FED ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_fed | nStart: 6 | nEns: 40/

In [343]:
comp_ens_anom('sfc_fed', 'monthly', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_FED_ENSEMBLE/sfc_fed_ens_global_monthly_anom.nc


In [344]:
comp_ens_anom('sfc_fed', 'monthly', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_FED_ENSEMBLE/sfc_fed_ens_so_monthly_anom.nc


In [27]:
comp_ppp('sfc_fed', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_FED/sfc_fed_ts_global_monthly_ppp.nc


In [28]:
comp_ppp('sfc_fed', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_FED/sfc_fed_ts_so_monthly_ppp.nc


# SFC_IRR

In [150]:
# pcs.dir_inspect(rootdir+'SFC_IRR')

In [45]:
sfc_irr = xr.open_dataset(rootdir+'SFC_IRR/sfc_irr_0001_0300.nc')

In [47]:
comp_gc_metrics(sfc_irr, 'sfc_irr', resize=False, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_global_monthly_var.nc


In [48]:
comp_gc_metrics(sfc_irr, 'sfc_irr', resize=True, save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_so_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_gc_so_monthly_var.nc


In [146]:
comp_annual_mean(sfc_irr, 'sfc_irr', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_global_annual_mean.nc


In [147]:
comp_annual_mean(sfc_irr, 'sfc_irr', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_so_annual_mean.nc


In [180]:
comp_monthly_mean(sfc_irr, 'sfc_irr', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_global_monthly_mean.nc


In [181]:
comp_monthly_mean(sfc_irr, 'sfc_irr', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_so_monthly_mean.nc


In [230]:
comp_monthly_anom('sfc_irr', 'global', save=True)

Source: sfc_irr_ts_global_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_global_monthly_anom.nc


In [231]:
comp_monthly_anom('sfc_irr', 'so', save=True)

Source: sfc_irr_ts_so_monthly_mean.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_so_monthly_anom.nc


In [269]:
comp_monthly_var('sfc_irr', 'global', save=True)

Source: sfc_irr_ts_global_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_global_monthly_var.nc


In [270]:
comp_monthly_var('sfc_irr', 'so', save=True)

Source: sfc_irr_ts_so_monthly_anom.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_so_monthly_var.nc


In [282]:
pcs.comp_clim('sfc_irr', 'global', save=True)
pcs.comp_clim('sfc_irr', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_global_clim.nc
/home/bbuchovecky/storage/so_predict_derived/CTRL/SFC_IRR/sfc_irr_ts_so_clim.nc


In [321]:
comp_ens_global_mean([['SFC_IRR','sfc_irr','sfc_irr']], 'monthly', save=True)


######## SFC_IRR ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_I

In [322]:
comp_ens_so_mean([['SFC_IRR','sfc_irr','sfc_irr']], 'monthly', save=True)


######## SFC_IRR ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 40/

In [323]:
comp_ens_global_mean([['SFC_IRR','sfc_irr','sfc_irr']], 'annual', save=True)


######## SFC_IRR ########

this_ens.coords = ['nT']
this_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['Global']

oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 40/40

all_ens.coords = ['nT', 'nEns', 'nStart']
all_ens.data_vars = ['Global']

/home/bbuchovecky/storage/so_predict_derived/SFC_I

In [324]:
comp_ens_so_mean([['SFC_IRR','sfc_irr','sfc_irr']], 'annual', save=True)


######## SFC_IRR ########

this_ens.coords = ['nT']
this_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 1 | nEns: 40/40

these_ens.coords = ['nT', 'nEns']
these_ens.data_vars = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 2 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 3 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 4 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 5 | nEns: 40/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 20/40
oooooooooooooooooooo
var: sfc_irr | nStart: 6 | nEns: 40/

In [345]:
comp_ens_anom('sfc_irr', 'monthly', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_IRR_ENSEMBLE/sfc_irr_ens_global_monthly_anom.nc


In [346]:
comp_ens_anom('sfc_irr', 'monthly', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/SFC_IRR_ENSEMBLE/sfc_irr_ens_so_monthly_anom.nc


In [29]:
comp_ppp('sfc_irr', 'global', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_IRR/sfc_irr_ts_global_monthly_ppp.nc


In [30]:
comp_ppp('sfc_irr', 'so', save=True)

/home/bbuchovecky/storage/so_predict_derived/PPP/SFC_IRR/sfc_irr_ts_so_monthly_ppp.nc
